In [3]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import sys
import pickle
import datetime
sys.path.append('/home/jupyter/bastianberger/inria1358')
from ML.preprocessing import create_datasets
from ML.model import make_model
import pandas as pd

In [5]:
patches_path = '/home/jupyter/bastianberger/inria1358/raw_data/patches2'

In [6]:
test_batches, train_batches, val_batches = create_datasets(patches_path, data_size=0.1)

2023-09-22 16:37:45.030782: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-22 16:37:45.251469: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-22 16:37:45.254651: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-22 16:37:45.263401: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild

✅ Test_batches with 11250 images (176 batches) created
✅ Train_batches with 7087 images (111 batches) created
✅ Val_batches with 3038 images (48 batches) created


In [ ]:
model_unet_point1, history_unet_point1 = make_model(train_batches, val_batches, input_shape=(200,200,3), model_name='unet', epochs=100)

✅ UNET Model initialized
✅ Model compiled
Epoch 1/100


2023-09-22 16:39:49.922923: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200
2023-09-22 16:40:08.205280: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.85GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-09-22 16:40:08.205341: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.85GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-09-22 16:40:08.569144: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.51GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if m

 94/111 [========================>.....] - ETA: 19s - loss: 0.4066 - accuracy: 0.8465 - mean_io_u_1: 0.4216

2023-09-22 16:42:07.242415: W tensorflow/core/kernels/data/cache_dataset_ops.cc:296] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


110/111 [============================>.] - ETA: 1s - loss: 0.3940 - accuracy: 0.8490 - mean_io_u_1: 0.4208

2023-09-22 16:42:34.508896: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-09-22 16:42:34.508979: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


111/111 [==============================] - 195s 1s/step - loss: 0.3934 - accuracy: 0.8491 - mean_io_u_1: 0.4208 - val_loss: 0.3059 - val_accuracy: 0.8604 - val_mean_io_u_1: 0.4222
Epoch 2/100
111/111 [==============================] - 151s 1s/step - loss: 0.2989 - accuracy: 0.8724 - mean_io_u_1: 0.4208 - val_loss: 0.2939 - val_accuracy: 0.8748 - val_mean_io_u_1: 0.4222
Epoch 3/100
111/111 [==============================] - 151s 1s/step - loss: 0.2740 - accuracy: 0.8862 - mean_io_u_1: 0.4208 - val_loss: 0.2540 - val_accuracy: 0.8968 - val_mean_io_u_1: 0.4233
Epoch 4/100
111/111 [==============================] - 151s 1s/step - loss: 0.2598 - accuracy: 0.8937 - mean_io_u_1: 0.4208 - val_loss: 0.2784 - val_accuracy: 0.8836 - val_mean_io_u_1: 0.4222
Epoch 5/100
111/111 [==============================] - 151s 1s/step - loss: 0.2511 - accuracy: 0.8982 - mean_io_u_1: 0.4208 - val_loss: 0.2503 - val_accuracy: 0.8951 - val_mean_io_u_1: 0.4224
Epoch 6/100
111/111 [==============================]

In [ ]:
hist_df = pd.DataFrame(history_unet_point1.history)
hist_df.to_csv('./logs/histories/unet_point1.csv')
model_unet_point1.save('.logs/models/unet_point1.keras')

In [11]:
pd.read_csv('./logs/histories/unet_point1.csv')

,Unnamed: 0,loss,accuracy,mean_io_u_1,val_loss,val_accuracy,val_mean_io_u_1
0,0,0.393351,0.849080,0.420775,0.305916,0.860398,0.422161
1,1,0.298917,0.872365,0.420775,0.293903,0.874807,0.422168
2,2,0.274030,0.886187,0.420775,0.254023,0.896829,0.423341
3,3,0.259818,0.893706,0.420775,0.278415,0.883573,0.422176
4,4,0.251109,0.898171,0.420775,0.250251,0.895070,0.422361
5,5,0.240798,0.903103,0.420775,0.235326,0.905231,0.421864
6,6,0.233087,0.906246,0.420775,0.231087,0.908614,0.421371
7,7,0.217962,0.912880,0.420775,0.211268,0.915719,0.422976
8,8,0.213215,0.915348,0.420775,0.208404,0.916180,0.421225
9,9,0.209126,0.916607,0.420775,0.215834,0.915227,0.422285


In [54]:
%tensorboard --logdir logs/fit